In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn. neighbors import LocalOutlierFactor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures

from scipy.stats import expon, reciprocal
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier
from missingpy import MissForest

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/envs/AI_dev/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv('./data/galaxy_final.csv', index_col=0)
X = df.drop('sold', axis=1)
y = df.sold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11,
                                                       stratify=y, shuffle=True)

# XGBoost Tuning
- https://www.kaggle.com/lifesailor/xgboost
- https://brunch.co.kr/@snobberys/137
- https://xzz201920.medium.com/xgbosst-booster-gbtree-v-s-dart-v-s-gblinear-82d8fcbb07d2

General Parameter
- booster: tree 기반 모델 / 선형 모델
    - https://xzz201920.medium.com/xgbosst-booster-gbtree-v-s-dart-v-s-gblinear-82d8fcbb07d2
    - skip_drop(default = 0, range [0, 1]) is the probability of skipping dropout. It has a higher priority than other DART parameters.
        - If skip_drop = 1, the dropout procedure would be skipped and dart is the same as gbtree.
    - If skip_drop≠0, rate_drop (default = 0, range [0, 1]) will drop a fraction of the trees before the model update in every iteration.
        - dropout makes dart between gbtree and random forest: “If no tree is dropped, dart is the same as (gbtree); if all the trees are dropped, dart is no different than random forest.”
- silent: 메세지 조절
- nthread: 병렬 처리 조절

Boost Parameter
- eta: Learning rate(일반적으로 0.01 - 0.2)
- min_child_weight: min_child_weight를 기준으로 추가 분기 결정(크면 Underfitting)
- max_depth: Tree 깊이 수
- max_leaf_node: 하나의 트리에서 node 개수
- gamma: split 하기 위한 최소의 loss 감소 정의
- subsample: 데이터 중 샘플링(0.5 - 1)
- colsample_bytree: column 중 sampling(0.5 - 1)
- colsample_bylevel: 각 level마다 샘플링 비율
- lambda: L2 nrom
- alpha: L1 norm
- scale_pos_weight: positive, negative weight 지정
- 기타 등

Learning Task Parameter
- object: 목적함수 종류
    - binary:logistic(이진 분류)
    - multi:softmax(다중 분류)
    - multi-softprob(다중 확률)
- eval_metric: 평가 지표
    - rmse – root mean square error
    - mae – mean absolute error
    - logloss – negative log-likelihood
    - error – Binary classification error rate (0.5 threshold)
    - merror – Multiclass classification error rate
    - mlogloss – Multiclass logloss
    - auc: Area under the curve
seed

### Overview
- high learning rate(0.05 - 0.3)를 선택하고 이 학습률에 맞는 tree 개수를 선정한다.
- tree-specific parameter를 수정한다.
- max_depth, min_child_weight, gamma, subsample, colsample_bytree
- regularization parameter를 수정한다.
- 학습률을 낮추고 다시 반복한다.

### Initialization
초기값은 다음과 같이 선정한다.
- max_depth = 5: 보통 4-6 를 시작점으로 한다.
- min_child_weight = 1 : 향후에 튜닝할 것이다.
- gamma = 0 : 0.1 - 0.2로 시작해도 된다. 그런데 어짜피 튜닝할 것이다.
- subsample, colsample_bytree = 0.8 : 보통 0.5 - 0.9로 시작한다.
- scale_pos_weight = 1: Because of high class imbalance.


### 1-1. Scaler + PolynomialFeatures

In [4]:
pipe1_1 = Pipeline([
                ('scale', MinMaxScaler()),
                ('poly', PolynomialFeatures()),
                ('classifier', XGBClassifier())
                ])

param_grid1_1 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[1, 2, 3]
              }
             ] 
grid1_1 = GridSearchCV(pipe1_1, param_grid1_1, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_1.fit(X_train, y_train)
print(grid1_1.best_params_)
print(grid1_1.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 12.2min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### 1-2. Scaler Only

In [6]:
pipe1_2 = Pipeline([
                ('scale', MinMaxScaler()),
#                 ('poly', PolynomialFeatures()),
                ('classifier', XGBClassifier())
                ])

param_grid1_2 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
#                'poly':[PolynomialFeatures()],
#                'poly__degree':[1, 2, 3]
              }
             ] 
grid1_2 = GridSearchCV(pipe, param_grid1_2, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_2.fit(X_train, y_train)
print(grid1_2.best_params_)
print(grid1_2.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.8min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### 1-3. Scale -> Poly -> ReduceDim

- PCA & LDA
    - https://huidea.tistory.com/126
- LDA
    - https://yamalab.tistory.com/41
    - http://www.datamarket.kr/xe/board_oFxn34/26649
- TSNE
    - https://agiantmind.tistory.com/215
    - https://lovit.github.io/nlp/representation/2018/09/28/tsne/

In [12]:
pipe1_3 = Pipeline([
                ('scale', MinMaxScaler()),
                 ('poly', PolynomialFeatures()),
                ('reduce_dims', PCA()),
                ('classifier', XGBClassifier())
                ])

param_grid1_3 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree': [1, 2, 3],
               'reduce_dims' : [PCA(), LDA(), TSNE()],
               'reduce_dims__n_components' : [5, 7, 9, 11]
              }
             ] 
grid1_3 = GridSearchCV(pipe1_3, param_grid1_3, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_3.fit(X_train, y_train)
print(grid1_3.best_params_)
print(grid1_3.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 39.7min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 64.3min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### 1-4. Scale -> Poly -> Feature Selection

```python
len(PolynomialFeatures(3).fit_transform(X_train)[0, :]) = 560
```
- 280
- 140
- 70
- 35

In [11]:
pipe1_4 = Pipeline([
                ('scale', MinMaxScaler()),
                 ('poly', PolynomialFeatures()),
                ('feature_selection', RFE(LogisticRegression())),
                ('classifier', XGBClassifier())
                ])

param_grid1_4 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree': [1, 2, 3],
               'feature_selection' : [RFE(LogisticRegression()),
                                     RFE(RandomForestClassifier())],
               'feature_selection__n_features_to_select' : [280, 140, 70, 35]
#                'reduce_dims' : [PCA(), LDA(), TSNE()],
#                'reduce_dims__n_components' : [5, 7, 9, 11]
              }
             ]
grid1_4 = GridSearchCV(pipe1_4, param_grid1_4, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_4.fit(X_train, y_train)
print(grid1_4.best_params_)
print(grid1_4.best_score_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 47.3min
[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed: 162.7min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### Scaler+RFE+Scaler + Poly > Scaler Only > Scaler + Poly + DimReduction
- 0.808 > 0.805 > 0.797 > 0.789


### max_depth, min_child_weight

In [7]:
# grid1_3 base (scaler-poly-xgb)
# max_depth와 min_child_weight를 튜닝한다.

pipe = Pipeline([
                ('scale', MinMaxScaler()),
                ('poly', PolynomialFeatures()),
                ('classifier', XGBClassifier())
                ])

param_grid2 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':range(3,10,3),
               'classifier__min_child_weight':range(1,6,2),
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid2 = GridSearchCV(pipe, param_grid2, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid2.fit(X_train, y_train)
print(grid2.best_params_)
print(grid2.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 15.9min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

In [13]:
# grid1_4 base (scaler-poly-rfe-xgb)
# max_depth와 min_child_weight를 튜닝한다.

pipe_1_4 = Pipeline([
                ('scale', MinMaxScaler()),
                ('poly', PolynomialFeatures()),
                ('feature_selection', RFE(RandomForestClassifier())),
                ('classifier', XGBClassifier())
                ])

param_grid2_1 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':range(3,10,3),
               'classifier__min_child_weight':range(1,6,2),
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3],
               'feature_selection' : [RFE(RandomForestClassifier())],
               'feature_selection__n_features_to_select' : [140]
              }
             ]
grid2_1 = GridSearchCV(pipe_1_4, param_grid2_1, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid2_1.fit(X_train, y_train)
print(grid2_1.best_params_)
print(grid2_1.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

### Gamma

In [18]:
# grid1_3 base (scaler-poly-xgb)
# Gamma를 튜닝한다.
param_grid3 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[i/10.0 for i in range(0,10)],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid3 = GridSearchCV(pipe, param_grid3, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid3.fit(X_train, y_train)
print(grid3.best_params_)
print(grid3.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 11.9min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs':

In [ ]:
# grid1_4 base (scaler-poly-rfe-xgb)
# Gamma 튜닝

pipe_1_4 = Pipeline([
                ('scale', MinMaxScaler()),
                ('poly', PolynomialFeatures()),
                ('feature_selection', RFE(RandomForestClassifier())),
                ('classifier', XGBClassifier())
                ])

param_grid3_1 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth': [grid2_1.best_params_['classifier__max_depth']], # 
               'classifier__min_child_weight': [grid2_1.best_params_['classifier__min_child_weight']], 
               'classifier__gamma':[i/10.0 for i in range(0,10)],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3],
               'feature_selection' : [RFE(RandomForestClassifier())],
               'feature_selection__n_features_to_select' : [140]
              }
             ]
grid3_1 = GridSearchCV(pipe_1_4, param_grid3_1, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid3_1.fit(X_train, y_train)
print(grid3_1.best_params_)
print(grid3_1.best_score_)

### subsample, colsample_bytree

In [20]:
# subsample and colsample_bytree를 튜닝한다.
param_grid4 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[i/10.0 for i in range(6,10)],
               'classifier__colsample_bytree':[i/10.0 for i in range(6,10)],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid4 = GridSearchCV(pipe, param_grid4, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid4.fit(X_train, y_train)
print(grid4.best_params_)
print(grid4.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 19.2min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.7, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.9, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs':

### n_estimators

In [21]:
# n_estimators 튜닝

param_grid5 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[50, 100, 300, 500, 750, 1000],
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[grid4.best_params_['classifier__subsample']], # 0.7
               'classifier__colsample_bytree':[grid4.best_params_['classifier__colsample_bytree']], # 0.9
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid5 = GridSearchCV(pipe, param_grid5, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid5.fit(X_train, y_train)
print(grid5.best_params_)
print(grid5.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.8min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.7, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.9, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs':

In [28]:
# rate_drop, skip_drop 튜닝
param_grid6 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': np.arange(0.1, 0.55, 0.05),
              'classifier__skip_drop': np.arange(0.1, 0.55, 0.05),
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[grid5.best_params_['classifier__n_estimators']], # 1000
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[grid4.best_params_['classifier__subsample']], # 0.7
               'classifier__colsample_bytree':[grid4.best_params_['classifier__colsample_bytree']], # 0.9
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid6 = GridSearchCV(pipe, param_grid6, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid6.fit(X_train, y_train)
print(grid6.best_params_)
print(grid6.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 47.9min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed: 96.6min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.3500000000000001, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021,
              skip_drop=0.30000000000000004, subsample=0.7, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.9, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators'

In [29]:
# learning rate

param_grid7 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [grid6.best_params_['classifier__rate_drop']],
              'classifier__skip_drop': [grid6.best_params_['classifier__skip_drop']],
             'classifier__learning_rate': [0.01, 0.03, 0.05, 0.1],
             'classifier__n_estimators':[grid5.best_params_['classifier__n_estimators']], # 1000
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[grid4.best_params_['classifier__subsample']], # 0.7
               'classifier__colsample_bytree':[grid4.best_params_['classifier__colsample_bytree']], # 0.9
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ]
grid7 = GridSearchCV(pipe, param_grid7, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid7.fit(X_train, y_train)
print(grid7.best_params_)
print(grid7.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  5.4min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.3500000000000001, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021,
              skip_drop=0.30000000000000004, subsample=0.7, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.9, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators'

In [15]:
# fit - early stopping rounds
#xgb_best = grid7.best_estimator_[2]
xgb_best = XGBClassifier(booster='dart', colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, importance_type='gain', learning_rate=0.1,
              max_depth=3, min_child_weight=1,
              n_estimators=1000, n_jobs=-1,
              nthread=-1, objective='binary:logistic',
              rate_drop=0.3500000000000001, 
              scale_pos_weight=1, seed=2021,
              skip_drop=0.30000000000000004, subsample=0.7)

best_esr_stoprounds = (-1, -1)
for i, esr in enumerate(np.arange(50, 550, 50)):
    X_train_c, X_test_c, y_train_c, y_test_c = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
    X_train_c, X_valid_c, y_train_c, y_valid_c = train_test_split(X_train_c, y_train_c,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    stratify=y_train_c,
                                                                    random_state=11)
    
    scaler = MinMaxScaler()
    poly = PolynomialFeatures(degree=3)
    
    X_train_c = scaler.fit_transform(X_train_c)
    X_valid_c = scaler.transform(X_valid_c)
    X_test_c = scaler.transform(X_test_c)
    
    X_train_c = poly.fit_transform(X_train_c)
    X_valid_c = poly.transform(X_valid_c)
    X_test_c = poly.transform(X_test_c)
    
    xgb_best.fit(X_train_c, y_train_c, early_stopping_rounds=esr, eval_metric="error",
                 eval_set=[(X_train_c, y_train_c), (X_valid_c, y_valid_c)], verbose=0)
    acc = accuracy_score(y_test, xgb_best.predict(X_test_c))
    
    if acc > best_esr_stoprounds[1]:
        best_esr_stoprounds = (esr, acc)
        print(i, best_esr_stoprounds)

KeyboardInterrupt: 

In [85]:
best_esr_stoprounds

(50, 0.8383838383838383)

In [17]:
xgb_best = XGBClassifier(booster='dart', colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, importance_type='gain', learning_rate=0.1,
              max_depth=3, min_child_weight=1,
              n_estimators=1000, n_jobs=-1,
              nthread=-1, objective='binary:logistic',
              rate_drop=0.3500000000000001, 
              scale_pos_weight=1, seed=2021,
              skip_drop=0.30000000000000004, subsample=0.7)

best_esr_stoprounds_rfe = (-1, -1)
for i, esr in enumerate(np.arange(50, 550, 50)):
    X_train_c, X_test_c, y_train_c, y_test_c = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
    X_train_c, X_valid_c, y_train_c, y_valid_c = train_test_split(X_train_c, y_train_c,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    stratify=y_train_c,
                                                                    random_state=11)
    
    scaler = MinMaxScaler()
    poly = PolynomialFeatures(degree=3)
    rfe = RFE(RandomForestClassifier(), n_features_to_select=140)
    
    X_train_c = scaler.fit_transform(X_train_c)
    X_valid_c = scaler.transform(X_valid_c)
    X_test_c = scaler.transform(X_test_c)
    
    X_train_c = poly.fit_transform(X_train_c)
    X_valid_c = poly.transform(X_valid_c)
    X_test_c = poly.transform(X_test_c)
    
    X_train_c = rfe.fit_transform(X_train_c, y_train_c)
    X_valid_c = rfe.transform(X_valid_c)
    X_test_c = rfe.transform(X_test_c)
    
    xgb_best.fit(X_train_c, y_train_c, early_stopping_rounds=esr, eval_metric="error",
                 eval_set=[(X_train_c, y_train_c), (X_valid_c, y_valid_c)], verbose=0)
    acc = accuracy_score(y_test, xgb_best.predict(X_test_c))
    
    if acc > best_esr_stoprounds_rfe[1]:
        best_esr_stoprounds_rfe = (esr, acc)
        print(i, best_esr_stoprounds_rfe)

0 (50, 0.8282828282828283)


KeyboardInterrupt: 

In [ ]:
best_esr_stoprounds_rfe

# Feature Selection

In [3]:
scaler = MinMaxScaler()
poly = PolynomialFeatures(degree=3)

X_train_c, X_test_c, y_train_c, y_test_c = X_train.copy(), X_test.copy(), y_train.copy(), y_test.copy()
X_train_c = scaler.fit_transform(X_train_c)
X_test_c = scaler.transform(X_test_c)
X_train_p = poly.fit_transform(X_train_c)
X_test_p = poly.transform(X_test_c)

In [99]:
len(X_train_c[0, :]), len(X_train_p[0, :])

(13, 560)

In [4]:
from sklearn.ensemble import ExtraTreesClassifier

etc_model = ExtraTreesClassifier()
etc_model.fit(X_train_p, y_train_c)
etc_model.
#print(etc_model.feature_importances_)
feature_list = pd.concat([pd.Series(X.columns), pd.Series(etc_model.feature_importances_)], axis=1)
feature_list.columns = ['features_name', 'importance']
feature_list.sort_values("importance", ascending =False)[:30]

,features_name,importance
2,startprice,0.039908
27,NaN,0.026178
196,NaN,0.020621
16,NaN,0.019841
3,productSeries_imputed,0.016792
38,NaN,0.015045
207,NaN,0.013881
29,NaN,0.013553
107,NaN,0.012192
118,NaN,0.011159


In [5]:
model = LogisticRegression()
rfe = RFE(model, 20)
fit = rfe.fit(X_train_p, y_train_c)

print(f"Num Features: {fit.n_features_}")
print(f"Selected features: {X[:,fit.support_]}")
print(f"Feature Ranking: {fit.ranking_}")

Num Features: 20


TypeError: '(slice(None, None, None), array([False, False, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
        True, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, Fal

In [6]:
X_train_p.shape, rfe.transform(X_train_p).shape

((1188, 560), (1188, 20))

In [115]:
print(f"Num Features: {fit.n_features_}")
#print(f"Selected features: {X[:, fit.support_]}")
print(f"Feature Ranking: {fit.ranking_}")

Num Features: 20
Feature Ranking: [459  10   3   1   1 401 380 275 377 248 305  22  56 389 283   1   1  71
 450 325 441 114  78  97 477 219 429   1   1 301 416 252  63 357 175 411
 196 197  58 117 458 431   1  36  45   1 206   8  55  80  15 355 406 419
   1 360 310 229 332 304 448 253  53 473 290 204 225 165 447  33 113 386
 145  34 257 268 444 184  62 465 494 115 222 430 337 452 394 190 480 158
 354 498 151 511 286 361 522 439 453 228 330 478 139 538 132  40  67  65
   9 141  72  37 217  79 417 128  11 284 168   4  14 369 328 334 216 322
 314   1  44 368 230 289  95  29 485 212 246 313  26 155   1 126 317  70
  69 194 356  96 127 154 131 471  51  66 482 466  50 407 449 162 238 481
  38   1 507 104 220 475 462 182 255 135 442 489 134 281 265 446 315  82
 474 174 245 432 179 285 218 533 440 502 457 261 343 276 530 463   1   1
 338 342 348  94 259 202 306 129 235 183 318 207 391 170 266 346   1  83
   1  24  31 157   1 366 490 500 200 349  86  84  54 399 464   7 277 374
 352 156  25 339 

In [ ]:
model = RandomForestClassifier()
rfe = RFE(model, 5)
fit = rfe.fit(X_train_p, y_train_c)

print(f"Num Features: {fit.n_features_}")
print(f"Selected features: {X.columns[fit.support_]}")
print(f"Feature Ranking: {fit.ranking_}")

In [109]:
# Install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector
selector = SequentialFeatureSelector(LogisticRegression(), scoring='accuracy', 
                                     verbose=1, k_features=20, forward=False, n_jobs=-1)
selector.fit(X_train_p, y_train_c)
selected_f = selector.k_feature_names_

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   46.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 560 out of 560 | elapsed:  2.2min finished
Features: 559/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 559 out of 559 | elapsed:  1.9min finished
Features: 558/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   38.1s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 558 out of 5

# Local Outlier Factor

In [88]:
test_neighbors = np.linspace(1, 101, num=50).astype(int)
test_contams = np.linspace(0.01, 0.3, num=30)
best_params, best_acc, X2, y2 = 0, 0, 0, 0

def tune_lof_xgb(model, df, scaler=None, poly=None, dim_reduction=None, ref=None):
    best_params, best_acc, X2, y2 = 0, 0, 0, 0
    for i, tn in enumerate(test_neighbors):
        for j, tc in enumerate(test_contams):
            
            clf = LocalOutlierFactor(n_neighbors=tn, contamination=tc)
            y_pred = clf.fit_predict(df.drop('sold', axis=1))
            lof_outlier_idx = pd.Series(y_pred)[pd.Series(y_pred)==-1].index
            df_lof2 = df.drop(lof_outlier_idx)
            
            X2 = df_lof2.drop('sold', axis=1)
            y2 = df_lof2.sold
            X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    stratify=y2,
                                                                    random_state=11)
            X2_train, X2_valid, y2_train, y2_valid = train_test_split(X2_train, y2_train,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    stratify=y2_train,
                                                                    random_state=11)
            
            if scaler:
                X2_train = scaler.fit_transform(X2_train)
                X2_valid = scaler.transform(X2_valid)
                X2_test = scaler.transform(X2_test)
                
            if poly:
                X2_train = poly.fit_transform(X2_train)
                X2_valid = poly.transform(X2_valid)
                X2_test = poly.transform(X2_test)
            
            if dim_reduction:
                X2_train = dim_reduction.fit_transform(X2_train)
                X2_valid = dim_reduction.transform(X2_valid)
                X2_test = dim_reduction.transform(X2_test)
                
            if ref:
                X2_train = ref.fit_transform(X2_train)
                X2_valid = ref.transform(X2_valid)
                X2_test = ref.transform(X2_test)
            
            mod = model
            mod.fit(X2_train, y2_train, early_stopping_rounds=50, eval_metric="error",
                 eval_set=[(X2_train, y2_train), (X2_valid, y2_valid)], verbose=0)
            mod_acc = accuracy_score(y2_test, mod.predict(X2_test))
            if best_acc < mod_acc:
                best_acc = mod_acc
                best_params = (tn, tc)
                X2 = X2
                y2 = y2
                print((i, j, tn, tc, best_acc))
    
    return best_params, best_acc, X2, y2

In [ ]:
xgb_best = XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.3500000000000001, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021,
              skip_drop=0.30000000000000004, subsample=0.7, tree_method=None,



In [89]:
xgb_best = grid7.best_params_['classifier']
xgb_scaler = MinMaxScaler()
xgb_poly = PolynomialFeatures(degree=3)
xgb_rfe = RFE(RandomForestClassifier(),
              n_features_to_select=140)
xgb_lof_tune = tune_lof_xgb(xgb_best, df,
                                  scaler=xgb_scaler,
                                  poly=xgb_poly)
xgb_lof_tune[:2]

(0, 0, 1, 0.01, 0.8129251700680272)
(0, 1, 1, 0.019999999999999997, 0.8178694158075601)
(0, 2, 1, 0.03, 0.8194444444444444)
(0, 4, 1, 0.049999999999999996, 0.8368794326241135)
(3, 6, 7, 0.06999999999999999, 0.8375451263537906)
(3, 8, 7, 0.08999999999999998, 0.8413284132841329)
(4, 28, 9, 0.29, 0.8483412322274881)
(9, 22, 19, 0.22999999999999998, 0.851528384279476)
(18, 24, 37, 0.24999999999999997, 0.852017937219731)
(29, 26, 60, 0.26999999999999996, 0.8617511520737328)


((60, 0.26999999999999996), 0.8617511520737328)